IMPORTANT: Notebook based on the [original ReFair code](https://anonymous.4open.science/r/ReFAIR-Toward-a-Context-Aware-Fairness-Recommender-in-Requirement-Engineering-18C7/README.md).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
import fasttext
from lazypredict.Supervised import LazyClassifier
from transformers import BertTokenizer
import gensim
import gensim.downloader

dataset = pd.read_excel("Synthetic User Stories.xlsx") # Change the dataset here
dataset

In [ ]:
target = []
for row in dataset.iterrows():
    target.append(np.where(dataset["Domain"].unique() == row[1]["Domain"])[0][0])
dataset["Target"] = target
dataset["Target"]

In [ ]:
def getTrainSetFastText():
    #ft_model = fasttext.load_model("fasttext_model.bin")
    ft_model = fasttext.load_model("cc.en.100.bin")
    traindata = []
    for msg in dataset['User Story']:
        traindata.append(ft_model.get_sentence_vector(msg))
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetTFIDF():
    countvec = CountVectorizer(max_features=100)
    bow = countvec.fit_transform(dataset['User Story']).toarray()
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(bow).toarray()
    training_data = pd.DataFrame(X)
    training_data.columns = training_data.columns.astype(str)
    return training_data

def getTrainSetBERT():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_data = tokenizer(dataset['User Story'].tolist(), padding=True, truncation=True, max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetWord2Vec():
    #w2v_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec-google-news-300.bin', binary=True)
    w2v_model = gensim.downloader.load('word2vec-google-news-300')
    traindata = []
    for msg in dataset['User Story']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in w2v_model:
                vecs.append(w2v_model[word][:100])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

def getTrainSetGlove():
    glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
    traindata = []
    for msg in dataset['User Story']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in glove_vectors:
                vecs.append(glove_vectors[word])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    return traindata

# FastText

In [ ]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)
X = getTrainSetFastText()
y = dataset['Target']
counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result.loc[counter]["Fold"] = foldcounter
            result.loc[counter]["Model"] = model[0]
            result.loc[counter]["Accuracy"] = round(model[1][0],3)
            result.loc[counter]["F1-Score"] = round(model[1][3],3)
            counter += 1
        foldcounter += 1
result

In [ ]:
result = result.dropna()

In [ ]:
result.to_excel("ResultFastText_domain.xlsx")

In [ ]:
average_scores = result.groupby('Model')[['Accuracy', 'F1-Score']].mean().reset_index()
average_scores

# TF-IDF

In [ ]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)
X = getTrainSetTFIDF() 
y = dataset['Target']
counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result.loc[counter]["Fold"] = foldcounter
            result.loc[counter]["Model"] = model[0]
            result.loc[counter]["Accuracy"] = round(model[1][0],3)
            result.loc[counter]["F1-Score"] = round(model[1][3],3)
            counter += 1
        foldcounter += 1
result

In [ ]:
result = result.dropna()

In [ ]:
result.to_excel("ResultTF-IDF_domain.xlsx")

In [ ]:
average_scores = result.groupby('Model')[['Accuracy', 'F1-Score']].mean().reset_index()
average_scores

# BERT

In [ ]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)
X = getTrainSetBERT() 
y = dataset['Target']
counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result.loc[counter]["Fold"] = foldcounter
            result.loc[counter]["Model"] = model[0]
            result.loc[counter]["Accuracy"] = round(model[1][0],3)
            result.loc[counter]["F1-Score"] = round(model[1][3],3)
            counter += 1
        foldcounter += 1
result

In [ ]:
result = result.dropna()

In [ ]:
result.to_excel("ResultBERT_domain.xlsx")

In [ ]:
average_scores = result.groupby('Model')[['Accuracy', 'F1-Score']].mean().reset_index()
average_scores

# Word2Vec

In [ ]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)
X = getTrainSetWord2Vec() 
y = dataset['Target']
counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result.loc[counter]["Fold"] = foldcounter
            result.loc[counter]["Model"] = model[0]
            result.loc[counter]["Accuracy"] = round(model[1][0],3)
            result.loc[counter]["F1-Score"] = round(model[1][3],3)
            counter += 1
        foldcounter += 1
result

In [ ]:
result = result.dropna()

In [ ]:
result.to_excel("ResultWord2Vec_domain.xlsx")

In [ ]:
average_scores = result.groupby('Model')[['Accuracy', 'F1-Score']].mean().reset_index()
average_scores

# Glove

In [ ]:
result = pd.DataFrame(columns=["Fold","Model","Accuracy","F1-Score"], index=np.arange(300))
fold = KFold(n_splits=10, random_state=6666, shuffle=True)
X = getTrainSetGlove() 
y = dataset['Target']
counter = 0
foldcounter = 1
for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
            X[ X.index.isin(train_index)], X[ X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
        models,predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result.loc[counter]["Fold"] = foldcounter
            result.loc[counter]["Model"] = model[0]
            result.loc[counter]["Accuracy"] = round(model[1][0],3)
            result.loc[counter]["F1-Score"] = round(model[1][3],3)
            counter += 1
        foldcounter += 1
result

In [ ]:
result = result.dropna()

In [ ]:
result.to_excel("ResultGlove_domain.xlsx")

In [ ]:
average_scores = result.groupby('Model')[['Accuracy', 'F1-Score']].mean().reset_index()
average_scores